## SQL 子查询 和 临时表
到目前为止，您已经学习了很多关于使用SQL处理数据的知识。本课将集中讨论三个主题:
* 子查询
* 临时表达式
* 持续的派生表

In [1]:
import pandas as pd
import sqlite3 as sql

In [2]:
database = 'parchposey.db'
connection = sql.connect(database)

子查询和表表达式都是能够编写查询来创建表，然后编写查询来与这个新创建的表交互的方法。有时，当直接使用数据库中的现有表时，您试图回答的问题没有答案。

但是，如果我们能够从现有表创建新表，我们知道我们可以查询这些新表来回答我们的问题。这就是这一课的问题的救急之处。

如果您还想不出一个可能需要这样查询的问题，不要担心，因为您将看到一大堆这样的问题!

每当我们需要使用现有的表来创建一个新表，然后我们想再次查询这个新表时，这就意味着我们需要使用某种子查询。在接下来的几个概念中，我们将一起浏览一个示例。

## 子查询格式

#### 不建议的格式

SELECT * FROM (SELECT DATE_TRUNC('day',occurred_at) AS day, channel, COUNT(*) as events FROM web_events GROUP BY 1,2 ORDER BY 3 DESC) sub;

#### 建议的格式

SELECT * <br>
FROM ( <br>
SELECT DATE_TRUNC('day',occurred_at) AS day, <br>
channel, COUNT(*) as events <br>
FROM web_events <br>
GROUP BY 1,2 <br>
ORDER BY 3 DESC) sub; <br>

![](Picture/DB_outline.png)

## 练习1
提供**total_amt_usd**销售额最大的每个**region**的**sales_rep**的**name**。

对于这样的相对复杂的查询我们需要将逐步的构建子查询

首先，我想找到与每个销售代表相关联的total_amt_usd总数，我还想知道他们所在的地区。下面的查询提供了这些信息。

In [3]:
query =  "\
SELECT s.name rep_name, r.name region_name, SUM(o.total_amt_usd) total_amt \
FROM sales_reps s \
JOIN accounts a \
ON a.sales_rep_id = s.id \
JOIN orders o \
ON o.account_id = a.id \
JOIN region r \
ON r.id = s.region_id \
GROUP BY 1,2 \
ORDER BY 3 DESC \
LIMIT 10 \
"
df = pd.read_sql(query, connection)
df

,rep_name,region_name,total_amt
0,Earlie Schleusner,Southeast,1098137.72
1,Tia Amato,Northeast,1010690.60
2,Vernita Plump,Southeast,934212.93
3,Georgianna Chisholm,West,886244.12
4,Arica Stoltzfus,West,810353.34
5,Dorotha Seawell,Southeast,766935.04
6,Nelle Meaux,Southeast,749076.16
7,Sibyl Lauria,Northeast,722084.27
8,Maren Musto,Southeast,702697.29
9,Brandie Riva,West,675917.64


接下来，我找出每个区域的销售额的最大值。

In [4]:
query = "SELECT region_name, MAX(total_amt) total_amt \
            FROM (SELECT s.name rep_name, r.name region_name, SUM(o.total_amt_usd) total_amt \
                FROM sales_reps s \
                JOIN accounts a \
                ON a.sales_rep_id = s.id \
                JOIN orders o \
                ON o.account_id = a.id \
                JOIN region r \
                ON r.id = s.region_id \
                GROUP BY 1,2 ) t1 \
        GROUP BY region_name"
    
df = pd.read_sql(query, connection)
df

,region_name,total_amt
0,Midwest,675637.19
1,Northeast,1010690.60
2,Southeast,1098137.72
3,West,886244.12


最后我们需要将之前的这两个查询的表JOIN在一起，并且通过使**region** 和 **total_amt**匹配来找出对应的销售代表的**name**

In [10]:
query = "SELECT t3.rep_name, t3.region_name, t3.total_amt \
FROM(SELECT region_name, MAX(total_amt) total_amt \
     FROM(SELECT s.name rep_name, r.name region_name, SUM(o.total_amt_usd) total_amt \
             FROM sales_reps s \
             JOIN accounts a \
             ON a.sales_rep_id = s.id \
             JOIN orders o \
             ON o.account_id = a.id \
             JOIN region r \
             ON r.id = s.region_id \
             GROUP BY 1, 2) t1 \
     GROUP BY 1) t2 \
JOIN (SELECT s.name rep_name, r.name region_name, SUM(o.total_amt_usd) total_amt \
     FROM sales_reps s \
     JOIN accounts a \
     ON a.sales_rep_id = s.id \
     JOIN orders o \
     ON o.account_id = a.id \
     JOIN region r \
     ON r.id = s.region_id \
     GROUP BY 1,2 \
     ORDER BY 3 DESC) t3 \
ON t3.region_name = t2.region_name AND t3.total_amt = t2.total_amt;"
    
df = pd.read_sql(query, connection)
df

,rep_name,region_name,total_amt
0,Charles Bidwell,Midwest,675637.19
1,Tia Amato,Northeast,1010690.60
2,Earlie Schleusner,Southeast,1098137.72
3,Georgianna Chisholm,West,886244.12


![](Picture/DB_outline.png)

## 练习2
在total_amt_usd最高的地区，总共下了多少订单?

In [13]:
query = "\
    SELECT r.name region_name, SUM(o.total_amt_usd) total_amt \
    FROM sales_reps s \
    JOIN accounts a \
    ON a.sales_rep_id = s.id \
    JOIN orders o \
    ON o.account_id = a.id \
    JOIN region r \
    ON r.id = s.region_id \
    GROUP BY 1" 
df = pd.read_sql(query, connection)
df

,region_name,total_amt
0,Midwest,3013486.51
1,Northeast,7744405.36
2,Southeast,6458497.00
3,West,5925122.96


然后我们只需要这个表中最大的区域。我考虑了两种方法来得到这个数目。一种是使用子查询获取最大值。另一种方法是降序排列，只提取顶部的值。

In [19]:
query = "\
        SELECT MAX(total_amt) \
        FROM (SELECT r.name region_name, SUM(o.total_amt_usd) total_amt \
             FROM sales_reps s \
             JOIN accounts a \
             ON a.sales_rep_id = s.id \
             JOIN orders o \
             ON o.account_id = a.id \
             JOIN region r \
             ON r.id = s.region_id \
             GROUP BY r.name) sub;"
df = pd.read_sql(query, connection)
df

,MAX(total_amt)
0,7744405.36


最后，通过使最大订单数量相等我们可以找出地区。

In [18]:
query = "\
SELECT r.name, COUNT(o.total) total_orders \
FROM sales_reps s \
JOIN accounts a \
ON a.sales_rep_id = s.id \
JOIN orders o \
ON o.account_id = a.id \
JOIN region r \
ON r.id = s.region_id \
GROUP BY r.name \
HAVING SUM(o.total_amt_usd) = ( \
      SELECT MAX(total_amt) \
      FROM (SELECT r.name region_name, SUM(o.total_amt_usd) total_amt \
              FROM sales_reps s \
              JOIN accounts a \
              ON a.sales_rep_id = s.id \
              JOIN orders o \
              ON o.account_id = a.id \
              JOIN region r \
              ON r.id = s.region_id \
              GROUP BY r.name) sub);"
df = pd.read_sql(query, connection)
df

,name,total_orders
0,Northeast,2357


## 练习3
有多少**account**的总购买量超过了拥有最多**standard_qty**的账户的总购买量?
![](Picture/DB_outline.png)

首先，我们想找到拥有最标准数量的纸张的帐户。这里的查询将找到这个帐户以及总购买金额:

In [13]:
query = "\
    SELECT a.name account_name, SUM(o.standard_qty) total_std, SUM(o.total) total \
    FROM accounts a \
    JOIN orders o \
    ON o.account_id = a.id \
    GROUP BY 1 \
    ORDER BY 2 DESC \
    LIMIT 1; "
df = pd.read_sql(query, connection)
df

,account_name,total_std,total
0,Core-Mark Holding,41617,44750


接下来我们用上面的结果来找出所有总订单量大于Core-Mark Hloding的总订单量的**account**

In [12]:
query = "\
        SELECT a.name, sum(o.total) total \
        FROM orders o \
        JOIN accounts a \
        ON a.id = o.account_id \
        GROUP BY 1 \
        HAVING SUM(o.total) > (SELECT total \
                           FROM (SELECT a.name act_name, SUM(o.standard_qty) tot_std, SUM(o.total) total \
                                 FROM accounts a \
                                 JOIN orders o \
                                 ON o.account_id = a.id \
                                 GROUP BY 1 \
                                 ORDER BY 2 DESC \
                                 LIMIT 1) sub); "
df = pd.read_sql(query, connection)
df

,name,total
0,EOG Resources,56410
1,IBM,47506
2,Mosaic,49246


接下来只需要简单的使用**COUNT**计算一下数量就行了

In [16]:
query = "\
        SELECT COUNT(counter_tab.name) \
        FROM (SELECT a.name, SUM(o.total) total \
               FROM orders o \
               JOIN accounts a \
               ON a.id = o.account_id \
               GROUP BY 1 \
               HAVING SUM(o.total) > (SELECT total \
                           FROM (SELECT a.name act_name, SUM(o.standard_qty) tot_std, SUM(o.total) total \
                                 FROM accounts a \
                                 JOIN orders o \
                                 ON o.account_id = a.id \
                                 GROUP BY 1 \
                                 ORDER BY 2 DESC \
                                 LIMIT 1) inner_tab) \
                     ) counter_tab;"
df = pd.read_sql(query, connection)
df

,COUNT(counter_tab.name)
0,3


## 练习 4 
花费最多的客户(total_amt_usd 数量最多)，他在每个渠道中有多少web_events ?

首先要找到这个花费最多的土豪客户

In [20]:
query = "\
        SELECT a.id, a.name, SUM(o.total_amt_usd) tot_spent \
        FROM orders o \
        JOIN accounts a \
        ON a.id = o.account_id \
        GROUP BY a.id, a.name \
        ORDER BY 3 DESC \
        LIMIT 1;"
df = pd.read_sql(query, connection)
df

,id,name,tot_spent
0,4211,EOG Resources,382873.3


现在，通过匹配id，我们就可以算出该公司在每个频道上拥有的事件数量

In [18]:
query = "\
        SELECT a.name, w.channel, COUNT(*) \
        FROM accounts a \
        JOIN web_events w \
        ON a.id = w.account_id AND a.id =  (SELECT id \
                             FROM (SELECT a.id, a.name, SUM(o.total_amt_usd) tot_spent \
                                   FROM orders o \
                                   JOIN accounts a \
                                   ON a.id = o.account_id \
                                   GROUP BY a.id, a.name \
                                   ORDER BY 3 DESC \
                                   LIMIT 1) inner_table) \
        GROUP BY 1, 2 \
        ORDER BY 3 DESC;"
df = pd.read_sql(query, connection)
df

,name,channel,COUNT(*)
0,EOG Resources,direct,44
1,EOG Resources,organic,13
2,EOG Resources,adwords,12
3,EOG Resources,facebook,11
4,EOG Resources,twitter,5
5,EOG Resources,banner,4


练习章节中也准备了几道题，去试着做一下吧。加油

练习 section 10 的 5 6

## WITH
**WITH**语句通常被称为公共表表达式(CTE)。WITH语句可以把子表的创建从复杂的查询语句中脱离出来，并放到整个查询语句的最前面。尽管这些表达式的用途与子查询完全相同，但它们在实践中更为常见，因为它们对于以后的读者更容易理解逻辑。

![](Picture/WITH_template.png)

下面我们通过上面的练习4来了解一下WITH语句和子查询的异同

问：计算出每个频道每天的平均事件数。

通过子查询的方式

In [27]:
query = "\
        SELECT a.name, w.channel, COUNT(*) \
        FROM accounts a \
        JOIN web_events w \
        ON a.id = w.account_id AND a.id =  (SELECT id \
                             FROM (SELECT a.id, a.name, SUM(o.total_amt_usd) tot_spent \
                                   FROM orders o \
                                   JOIN accounts a \
                                   ON a.id = o.account_id \
                                   GROUP BY a.id, a.name \
                                   ORDER BY 3 DESC \
                                   LIMIT 1) inner_table) \
        GROUP BY 1, 2 \
        ORDER BY 3 DESC;"
df = pd.read_sql(query, connection)
df

,name,channel,COUNT(*)
0,EOG Resources,direct,44
1,EOG Resources,organic,13
2,EOG Resources,adwords,12
3,EOG Resources,facebook,11
4,EOG Resources,twitter,5
5,EOG Resources,banner,4


通过**WITH**语句的方式

我把之前子查询方式中的inner_table 提取出来并通过**WITH** 语句放在前面。

之后在后面调用的时候将inner_table 和 accounts， web_events 表JOIN 在一起来获得与子查询方式相同的结果。

In [30]:
query = "\
        WITH inner_table AS(SELECT a.id, a.name, SUM(o.total_amt_usd) tot_spent \
                                   FROM orders o \
                                   JOIN accounts a \
                                   ON a.id = o.account_id \
                                   GROUP BY a.id, a.name \
                                   ORDER BY 3 DESC \
                                   LIMIT 1) \
        SELECT a.name, w.channel, COUNT(*) \
        FROM accounts a \
        JOIN web_events w \
        ON a.id = w.account_id \
        JOIN inner_table \
        ON a.id = inner_table.id \
        GROUP BY 1, 2 \
        ORDER BY 3 DESC;"

df = pd.read_sql(query, connection)
df

,name,channel,COUNT(*)
0,EOG Resources,direct,44
1,EOG Resources,organic,13
2,EOG Resources,adwords,12
3,EOG Resources,facebook,11
4,EOG Resources,twitter,5
5,EOG Resources,banner,4


下面再做一些和with有关的练习

## 重复练习4. 提供total_amt_usd销售额最大的每个地区的sales_rep的名称。

In [63]:
query = " \
    WITH t1 AS ( \
   SELECT a.id, a.name, SUM(o.total_amt_usd) tot_spent \
   FROM orders o \
   JOIN accounts a \
   ON a.id = o.account_id \
   GROUP BY a.id, a.name \
   ORDER BY 3 DESC \
   LIMIT 1) \
SELECT a.name, w.channel, COUNT(*) \
FROM accounts a \
JOIN web_events w \
ON a.id = w.account_id AND a.id =  (SELECT id FROM t1) \
GROUP BY 1, 2 \
ORDER BY 3 DESC; "

df = pd.read_sql(query, connection)
df

,name,channel,COUNT(*)
0,EOG Resources,direct,44
1,EOG Resources,organic,13
2,EOG Resources,adwords,12
3,EOG Resources,facebook,11
4,EOG Resources,twitter,5
5,EOG Resources,banner,4


总结：

这节课是编写SQL的更高级序列的第一堂课。可以说，子查询和cte的高级功能是在公司的分析角色中使用最广泛的。能够将问题分解成必要的表，并使用生成的表找到解决方案在实践中是非常有用的。

如果你第一次没有得到这些问题的答案，不要害怕下次再来，再试一次。此外，你可以试着提出一些自己的问题，看看能否找到解决方案。

本课程的其余部分可能是某些分析角色的关键，但您现在已经涵盖了可能在日常基础上使用的所有主要SQL主题。

